In [1]:


import sys
import numpy as np
import pandas as pd
import rasterio
import refet
import pyproj as proj
import xarray
import requests
import multiprocessing as mp
import matplotlib.pyplot as plt
import configparser
import pathlib

In [2]:
#sys.path.append("//")
sys.path.append("../../../Micromet")
import micromet
#from micromet.volk import ffp_climatology as ffp
#from micromet import AmerifluxDataProcessor

In [3]:

# load initial flux data
#station = 'US-UTW'
#config_path = f'../../station_config/{station}.ini'
#config = configparser.ConfigParser()
#config.read(config_path)

spath = "../../secrets/config.ini"
sconfig = configparser.ConfigParser()
sconfig.read(spath)

ed_pass = sconfig['NLDAS']['pw']
ed_user = sconfig['NLDAS']['user']

In [6]:
import datetime

#nldas_out_dir = Path('C:/Users/paulinkenbrandt/Documents/GitHub/MicroMet/Notebooks/Footprints')


for date in pd.date_range(start='2021-01-01', end='2025-02-01',freq='h'):
    hour = date.hour
    micromet.download_nldas(date,
                           hour,
                           ed_user,
                           ed_pass,)

NLDAS_data\2021_01_01_00.nc already exists, not overwriting.
NLDAS_data\2021_01_01_01.nc already exists, not overwriting.
NLDAS_data\2021_01_01_02.nc already exists, not overwriting.
NLDAS_data\2021_01_01_03.nc already exists, not overwriting.
NLDAS_data\2021_01_01_04.nc already exists, not overwriting.
NLDAS_data\2021_01_01_05.nc already exists, not overwriting.
NLDAS_data\2021_01_01_06.nc already exists, not overwriting.
NLDAS_data\2021_01_01_07.nc already exists, not overwriting.
NLDAS_data\2021_01_01_08.nc already exists, not overwriting.
NLDAS_data\2021_01_01_09.nc already exists, not overwriting.
NLDAS_data\2021_01_01_10.nc already exists, not overwriting.
NLDAS_data\2021_01_01_11.nc already exists, not overwriting.
NLDAS_data\2021_01_01_12.nc already exists, not overwriting.
NLDAS_data\2021_01_01_13.nc already exists, not overwriting.
NLDAS_data\2021_01_01_14.nc already exists, not overwriting.
NLDAS_data\2021_01_01_15.nc already exists, not overwriting.
NLDAS_data\2021_01_01_16

In [7]:
micromet.clip_to_utah_merge(file_dir="./NLDAS_data/", years = [2021,2022,2023,2024])


2021
NLDAS_data\2021_01_01_00.nc
NLDAS_data\2021_01_01_01.nc
NLDAS_data\2021_01_01_02.nc
NLDAS_data\2021_01_01_03.nc
NLDAS_data\2021_01_01_04.nc
NLDAS_data\2021_01_01_05.nc
NLDAS_data\2021_01_01_06.nc
NLDAS_data\2021_01_01_07.nc
NLDAS_data\2021_01_01_08.nc
NLDAS_data\2021_01_01_09.nc
NLDAS_data\2021_01_01_10.nc
NLDAS_data\2021_01_01_11.nc
NLDAS_data\2021_01_01_12.nc
NLDAS_data\2021_01_01_13.nc
NLDAS_data\2021_01_01_14.nc
NLDAS_data\2021_01_01_15.nc
NLDAS_data\2021_01_01_16.nc
NLDAS_data\2021_01_01_17.nc
NLDAS_data\2021_01_01_18.nc
NLDAS_data\2021_01_01_19.nc
NLDAS_data\2021_01_01_20.nc
NLDAS_data\2021_01_01_21.nc
NLDAS_data\2021_01_01_22.nc
NLDAS_data\2021_01_01_23.nc
NLDAS_data\2021_01_02_00.nc
NLDAS_data\2021_01_02_01.nc
NLDAS_data\2021_01_02_02.nc
NLDAS_data\2021_01_02_03.nc
NLDAS_data\2021_01_02_04.nc
NLDAS_data\2021_01_02_05.nc
NLDAS_data\2021_01_02_06.nc
NLDAS_data\2021_01_02_07.nc
NLDAS_data\2021_01_02_08.nc
NLDAS_data\2021_01_02_09.nc
NLDAS_data\2021_01_02_10.nc
NLDAS_data\2021

In [8]:
micromet.calc_hourly_ffp_xr(input_data_dir=".", years = [2021,2022,2023,2024], output_dir="./output/")

2021
2022
2023
2024


In [11]:
micromet.extract_nldas_xr_to_df([2021,2022,2023,2024], input_path="./output/")

US-CdM
US-UTB
US-UTD
US-UTE
US-UTJ
US-UTM
US-UTN
US-UTP
US-UTV
US-UTW
US-CdM
US-UTB
US-UTD
US-UTE
US-UTJ
US-UTM
US-UTN
US-UTP
US-UTV
US-UTW
US-CdM
US-UTB
US-UTD
US-UTE
US-UTJ
US-UTM
US-UTN
US-UTP
US-UTV
US-UTW
US-CdM
US-UTB
US-UTD
US-UTE
US-UTJ
US-UTM
US-UTN
US-UTP
US-UTV
US-UTW


In [12]:
output_path = pathlib.Path("./output/")
eto_df = pd.read_parquet(output_path / 'nldas_all.parquet')
normed_df = micromet.normalize_eto_df(eto_df)
normed_df.to_parquet('nldas_all_normed.parquet')

In [17]:
eto_df = pd.read_parquet('nldas_all.parquet')

eto_df = eto_df.reset_index().rename(columns={'level_0':'year',
                                              'level_1':'stationid'})
eto_df['datetime'] = eto_df['datetime'] - pd.Timedelta(hours=7)
eto_df = eto_df.set_index(['stationid','datetime'])
eto_df.to_parquet('nldas_all.parquet')


In [45]:
def norm_minmax_dly_et(x):
    # Normalize using min-max scaling and then divide by the sum,
    # rounding to 4 decimal places.
    return np.round(((x - x.min()) / (x.max() - x.min())), 4)

def norm_dly_et(x):
    return np.round(x/x.sum(), 4)

def normalize_eto_df(df, eto_field='eto'):
    # Assuming eto_df has a datetime index or a 'datetime' column:
    df = eto_df.reset_index()
    df['date'] = df['datetime'].dt.date
    df['hour'] = df['datetime'].dt.hour

    # Define the desired hour range (inclusive start, exclusive end)
    start_hour = 6
    end_hour = 18

    # Filter the DataFrame for only the hours within the specified range
    mask = (df['hour'] >= start_hour) & (df['hour'] <= end_hour)
    df_subset = df.loc[mask].copy()

    # Apply the normalization for each station and each day
    df_subset['daily_min_max_ETo'] = df_subset.groupby(['stationid', 'date'])[eto_field].transform(norm_minmax_dly_et)
    df_subset['daily_ETo_normed'] = df_subset.groupby(['stationid', 'date'])['daily_min_max_ETo'].transform(norm_dly_et)
    df_subset = df_subset.set_index(['stationid','datetime'])

    df_final = pd.merge(eto_df,
                        df_subset[['daily_min_max_ETo','daily_ETo_normed']],
                        how='left',
                        left_index=True,
                        right_index=True,)
    #df_final = df.merge(df_subset[['datetime', 'daily_ETo_normed']], on='datetime', how='left')
    df_final['daily_ETo_normed'] = df_final['daily_ETo_normed'].fillna(0)
    return df_final
#df_final.to_parquet('nldas_all_normed.parquet')

In [44]:
df_final

year_x  level_2_x   eto_x   etr_x   pet_x  \
stationid datetime                                                         
US-CdM    2021-12-31 17:00:00    2022          0  0.0178  0.0211  0.0825   
          2021-12-31 18:00:00    2022          1  0.0112  0.0139  0.0052   
          2021-12-31 19:00:00    2022          2  0.0053  0.0073  0.0052   
          2021-12-31 20:00:00    2022          3  0.0000  0.0012  0.0052   
          2021-12-31 21:00:00    2022          4  0.0002  0.0015  0.0081   
...                               ...        ...     ...     ...     ...   
US-UTW    2024-12-31 12:00:00    2024       8779  0.1566  0.1995  0.1473   
          2024-12-31 13:00:00    2024       8780  0.1551  0.1980  0.1473   
          2024-12-31 14:00:00    2024       8781  0.1392  0.1814  0.1473   
          2024-12-31 15:00:00    2024       8782  0.1011  0.1358  0.0755   
          2024-12-31 16:00:00    2024       8783  0.0530  0.0800  0.0755   

                                 lwdown_x    swdown_x  temperature_x    rh_x  \
stationid datetime                                                             
US-CdM    2021-12-31 17:00:00  269.670013    4.400000     276.920013  0.0043   
          2021-12-31 18:00:00  269.679993    0.000000     275.959991  0.0042   
          2021-12-31 19:00:00  269.679993    0.000000     275.000000  0.0042   
          2021-12-31 20:00:00  277.220001    0.000000     274.040009  0.0042   
          2021-12-31 21:00:00  277.220001    0.000000     273.859985  0.0040   
...                                   ...         ...            ...     ...   
US-UTW    2024-12-31 12:00:00  182.779999  450.346985     271.929993  0.0020   
          2024-12-31 13:00:00  182.759995  437.032990     272.489990  0.0021   
          2024-12-31 14:00:00  184.889999  385.660004     273.049988  0.0022   
          2024-12-31 15:00:00  184.929993  293.871002     272.089996  0.0022   
          2024-12-31 16:00:00  184.960007  165.806000     271.140015  0.0021   

                                 pressure_x  ...    lwdown_y    swdown_y  \
stationid datetime                           ...                           
US-CdM    2021-12-31 17:00:00  80793.734375  ...  269.670013    4.400000   
          2021-12-31 18:00:00  80769.898438  ...  269.679993    0.000000   
          2021-12-31 19:00:00  80746.695312  ...         NaN         NaN   
          2021-12-31 20:00:00  80722.859375  ...         NaN         NaN   
          2021-12-31 21:00:00  80764.656250  ...         NaN         NaN   
...                                     ...  ...         ...         ...   
US-UTW    2024-12-31 12:00:00  83837.398438  ...  182.779999  450.346985   
          2024-12-31 13:00:00  83838.296875  ...  182.759995  437.032990   
          2024-12-31 14:00:00  83839.296875  ...  184.889999  385.660004   
          2024-12-31 15:00:00  83842.296875  ...  184.929993  293.871002   
          2024-12-31 16:00:00  83845.398438  ...  184.960007  165.806000   

                               temperature_y    rh_y    pressure_y  wind_y  \
stationid datetime                                                           
US-CdM    2021-12-31 17:00:00     276.920013  0.0043  80793.734375  6.0104   
          2021-12-31 18:00:00     275.959991  0.0042  80769.898438  4.9612   
          2021-12-31 19:00:00            NaN     NaN           NaN     NaN   
          2021-12-31 20:00:00            NaN     NaN           NaN     NaN   
          2021-12-31 21:00:00            NaN     NaN           NaN     NaN   
...                                      ...     ...           ...     ...   
US-UTW    2024-12-31 12:00:00     271.929993  0.0020  83837.398438  3.1438   
          2024-12-31 13:00:00     272.489990  0.0021  83838.296875  3.0410   
          2024-12-31 14:00:00     273.049988  0.0022  83839.296875  2.9373   
          2024-12-31 15:00:00     272.089996  0.0022  83842.296875  2.6350   
          2024-12-31 16:00:00     271.140015  0.0021  83845.398438  2.3277  